In [2]:
load("../Imports/GeneralMethods.sage", '../Imports/Utility.sage', '../Imports/Point.sage', '../Imports/Line.sage', '../Imports/TritangentPlane.sage', '../Imports/Group.sage', '../Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [3]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(QQ)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f) * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [4]:
# Define cubic SE3, find and classify the lines and compute the 45 tritangent planes

In [5]:
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE3 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)})

In [6]:
# Find simmetries

In [7]:
%time adm_SE3 = SE3.find_admissible_projectivities()
len(adm_SE3)

CPU times: user 680 ms, sys: 112 ms, total: 792 ms
Wall time: 2.63 s


108

In [8]:
%time simm_SE3 = SE3.find_simmetries(adm_SE3)
len(simm_SE3)

CPU times: user 395 ms, sys: 173 ms, total: 568 ms
Wall time: 735 ms


6

In [9]:
# Study how the simmetries permute the Eckardt points

In [10]:
Eck_perms = [Permutation(SE3.apply_proj_to_eck(proj[0])).to_permutation_group_element() for proj in simm_SE3]
Eck_perms_group = PermutationGroup(Eck_perms)
Eck_perms_group.order(), Eck_perms_group.structure_description()

(6, 'S3')

In [11]:
### geometric interpretation of the group of symmetries

In [12]:
## Action of the stabilizer of SE3 on the 27 lines:
Ga = perm_group_lines(SE3, simm_SE3)

## Action of the stabilizer of SE3 on the 45 tritangent planes:
Gb = perm_group_planes(SE3, simm_SE3)

print("Structure of the group Ga:")
print(PermutationGroup(Ga).structure_description())

print("")

print("Structure of the group Gb:")
print(PermutationGroup(Gb).structure_description())

Structure of the group Ga:
S3

Structure of the group Gb:
S3


In [13]:
## Action of the group Ga on the lines: computation of the orbits:
OGa = PermutationGroup(Ga).orbits()

## Action of the group on Gb the planes: computation of the orbits:
OGb = PermutationGroup(Gb).orbits()

In [14]:
# There are no fixed lines
[orb for orb in OGa if len(orb) == 1]

[]

In [16]:
## The orbits OGa are a partition of the set {1, 2, ..., 27}:
OGa

[[1, 9, 14],
 [2, 19, 10],
 [3, 22, 7, 13, 8, 4],
 [5, 25, 12, 21, 23, 17],
 [6, 26, 11, 20, 24, 16],
 [15, 18, 27]]

In [17]:
## We convert the elements of the orbits of OGa (which are numbers from 1 to 27) 
## into the corresponding lines expressed with the labels:
for orb in OGa:
    print([simbolic_line_from_index(i) for i in orb])

['E1', 'G3', 'F13']
['E2', 'F24', 'G4']
['E3', 'F34', 'G1', 'F12', 'G2', 'E4']
['E5', 'F45', 'G6', 'F26', 'F35', 'F16']
['E6', 'F46', 'G5', 'F25', 'F36', 'F15']
['F14', 'F23', 'F56']


In [33]:
## The group Ga permutes the elements of the first orbit of OGa (which has 3 elements)
## in all the 3! possibilities:
O1 = OGa[0]
totPerm = []
for gg in Ga:
    totPerm.append(tuple([gg(i) for i in O1]))

In [34]:
len(Set(totPerm))

6

In [35]:
print(totPerm)

[(1, 9, 14), (1, 14, 9), (14, 1, 9), (9, 14, 1), (14, 9, 1), (9, 1, 14)]


In [23]:
### hence OGa is S_3.